In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks'

 ai4bharat-IndicNER.ipynb	        eda_cleaning.ipynb
 Annotated_corpus.ipynb		        Hello4
'annotated dataset full.csv'	        indic-transformers-bn-bert.ipynb
'annotated dataset full new.csv'        Kowsher-bangla-bert.ipynb
 bert-base-multilingual-cased.ipynb     monsoon-nlp-bangla-electra.ipynb
 BERT-BILSTM-CRF-gcelikmasat.ipynb      ner_all_tokens_vocab.txt
'Bi-LSTM with CRF for NER.ipynb'        Ner_Bert1.ipynb
'BLNERC dataset.json'		        ner_dataset_sample.json
 BLNERC_dataset.json		        ner_token_id_name.pkl
 cache				       'NER using Random Forest and CRF.ipynb'
'Copy of Welcome to Colaboratory'       neuralspace-reverie
'Copy of Welcome to Colaboratory (1)'   sagorsarker-bangla-bert-base.ipynb
 csvtojson.ipynb		        Untitled0.ipynb
'dataset statistics.ipynb'	        Untitled2.ipynb


In [ ]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks")

In [ ]:
from datasets import load_metric
import numpy as np
metric = load_metric("seqeval")
import json

<ipython-input-8-9a9bb19245b7>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
import pickle

with open('ner_token_id_name.pkl', 'rb') as f:
    label_list = pickle.load(f)

In [ ]:
label_list

{0: 'O',
 1: 'B-PER',
 2: 'B-LOC',
 3: 'B-ORG',
 4: 'B-TIM',
 5: 'I-PER',
 6: 'I-LOC',
 7: 'I-ORG',
 8: 'I-TIM'}

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens = True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True, max_length=512)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    print(len(examples))
    return tokenized_inputs

## Load Dataset

In [ ]:
from datasets import Sequence

In [ ]:
from datasets import Dataset, load_dataset

In [ ]:
train_dataset = load_dataset("json", data_files={'train': "BLNERC dataset.json"}, field ='train', cache_dir="cache")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
train_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 23703
    })
})

In [ ]:
map(lambda x: None if x in none_items else x, train_dataset)

In [ ]:
test_dataset = load_dataset("json", data_files={'test': "BLNERC dataset.json"}, field ='test', cache_dir="cache")

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
validation_dataset = load_dataset("json", data_files={'validation': "BLNERC dataset.json"}, field ='validation', cache_dir="cache")

  0%|          | 0/1 [00:00<?, ?it/s]

## Make tokenizer

In [ ]:
def build_fast_bert_tokenizer(files, max_vocab_size):
    from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer

    from transformers import BertTokenizerFast, AutoTokenizer, DistilBertTokenizerFast
    assert isinstance(files, list)
    tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
    tokenizer.normalizer = normalizers.Sequence(
        [normalizers.NFD(), normalizers.Lowercase()]
    )
    tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()
    special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
    trainer = trainers.WordPieceTrainer(vocab_size=max_vocab_size, special_tokens=special_tokens)
    tokenizer.train(files=files, trainer=trainer)
    cls_token_id = tokenizer.token_to_id("[CLS]")
    sep_token_id = tokenizer.token_to_id("[SEP]")
    tokenizer.post_processor = processors.TemplateProcessing(
        single=f"[CLS]:0 $A:0 [SEP]:0",
        pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
        special_tokens=[
            ("[CLS]", cls_token_id),
            ("[SEP]", sep_token_id),
        ],
    )
    tokenizer.decoder = decoders.WordPiece(prefix="##")
    return BertTokenizerFast(tokenizer_object=tokenizer)

In [ ]:
tokenizer = build_fast_bert_tokenizer(["ner_all_tokens_vocab.txt"], 8000)

In [ ]:
tokenizer("মনির ঢাকায় থাকে").tokens()

['[CLS]', 'মনি', '##র', 'ঢাকা', '##য়', 'থাকে', '[SEP]']

In [ ]:
tokenizer.decode(tokenizer("b_ner on the way to train!!")["input_ids"])

'[CLS] [UNK] _ [UNK] [UNK] [UNK] [UNK] [UNK] [UNK]!! [SEP]'

In [ ]:
tokenizer("মনির ঢাকায় থাকে").tokens()

['[CLS]', 'মনি', '##র', 'ঢাকা', '##য়', 'থাকে', '[SEP]']

In [ ]:
len(tokenizer.vocab)

8000

## Data Collator

In [ ]:
batch_size = 16

In [ ]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
train_ds = train_dataset.map(tokenize_and_align_labels, batched=True)
test_ds = test_dataset.map(tokenize_and_align_labels, batched=True)
validation_ds = validation_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/23703 [00:00<?, ? examples/s]

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


Map:   0%|          | 0/2963 [00:00<?, ? examples/s]

2
2
2


Map:   0%|          | 0/2967 [00:00<?, ? examples/s]

2
2
2


In [ ]:
validation_dataset

DatasetDict({
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 2967
    })
})

## Train

In [ ]:
from transformers import TrainingArguments

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, BertForTokenClassification

model = BertForTokenClassification.from_pretrained("bert-base-multilingual-cased", num_labels=len(label_list))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

In [ ]:
pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
args = TrainingArguments(
    output_dir="Hello",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=1,
    report_to=[],
    save_steps=2000,
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds["train"],
    eval_dataset=validation_ds["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.186300,0.173216,0.711369,0.552722,0.622090,0.951932
2,0.010000,0.138355,0.755264,0.680560,0.715969,0.961834
3,0.137800,0.132047,0.734530,0.712597,0.723398,0.962446


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=4446, training_loss=0.1790445345655851, metrics={'train_runtime': 1150.5, 'train_samples_per_second': 61.807, 'train_steps_per_second': 3.864, 'total_flos': 2552474053777374.0, 'train_loss': 0.1790445345655851, 'epoch': 3.0})

## Tests

In [ ]:
trainer.evaluate(test_ds['test'])

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.13650551438331604,
 'eval_precision': 0.7530687071576598,
 'eval_recall': 0.6999555489702178,
 'eval_f1': 0.7255413914913225,
 'eval_accuracy': 0.9602828922765569,
 'eval_runtime': 15.2845,
 'eval_samples_per_second': 193.857,
 'eval_steps_per_second': 12.169,
 'epoch': 3.0}

In [ ]:
'''model.save_pretrained('bert-base-multilingual-cased/ner_model')
tokenizer.save_pretrained('bert-base-multilingual-cased/ner_tokenizer')'''

"model.save_pretrained('bert-base-multilingual-cased/ner_model')\ntokenizer.save_pretrained('bert-base-multilingual-cased/ner_tokenizer')"